In [62]:
# read in libraries
import pandas as pd
import numpy as np
import re
import os
import json
import sys
import datetime as dt
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
from tner import TransformersNER

# connect to sql
from mysql.connector import (connection)
from mysql.connector import Error

# load functions
with open('project_config.json','r') as fp: 
    project_config = json.load(fp)

module_path = os.path.join(project_config['project_module_relative_path'])
sys.path.append(module_path)

from ner.ner_processing import *




# SQL connection ------------------------------------------------------------------

In [63]:
# fetch values from environment variables and set the target database
hostname = os.environ['MYSQL_HOST']
username = os.environ['MYSQL_USER']
password = os.environ['MYSQL_PASSWORD']
dbname = 'found'

# establish connection to db1 database in your mysql service
cnx = connection.MySQLConnection(user=username,
                                 password=password,
                                 host=hostname,
                                 database=dbname)

# connect to sql
from mysql.connector import (connection)
from mysql.connector import Error

# fetch values from environment variables and set the target database
hostname = os.environ['MYSQL_HOST']
username = os.environ['MYSQL_USER']
password = os.environ['MYSQL_PASSWORD']
dbname = 'found'

# establish connection to db1 database in your mysql service
cnx = connection.MySQLConnection(user=username,
                                 password=password,
                                 host=hostname,
                                 database=dbname)


In [81]:
# query database ---------------------------------------------------------------------
query =("SELECT * FROM landmine_tweets WHERE date > '2023-04-02'")

#write to df and remove duplicates
df = pd.read_sql_query(query, cnx)
df = df.drop_duplicates(subset=['tweet', 'handle'], keep="first")
df.columns = df.columns.str.lower()


In [82]:
# detect language --------------------------------------------------------------------
nlp = en_core_web_sm.load()

# load spacy nlp model
nlp_model = spacy.load("en_core_web_sm")
Language.factory("language_detector", func=get_lang_detector)
nlp_model.add_pipe('language_detector', last=True)

# create a language column on dataframe
df["language"]= df["tweet"].apply(get_language)


In [83]:
# separate dataframe by eng and non eng tweets --------------------------------------------
# create dataframe for non english tweets
non_eng = df[df["language"] != "en"]

# write to csv
non_eng.to_csv("landmine_tweets_non_eng_processed.csv", index = False)

# create dataframe for english tweets
en = df[df["language"]=="en"]

# clean text on english tweets
en =  clean_tweets(en, 'tweet')

# write to csv
en.to_csv("landmine_tweets_eng_processed.csv", index = False)

# write back to main tweeets file
main = pd.read_csv("main.csv")
main = pd.concat([non_eng, en, main])
main['date']= main['date'].astype(str)
main = main.drop_duplicates(subset=['id', 'handle', 'date'])
#main.to_csv("main.csv", index = False)


src/ner/ner_processing.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[tweets] = df[tweets].str.lower()
src/ner/ner_processing.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[tweets] = df[tweets].map(lambda x: re.sub('http[s]?:\/\/[^\s]*', ' ', x))
src/ner/ner_processing.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pand

In [91]:
main.to_csv("main.csv", index = False)

In [37]:
#main = pd.concat([non_eng, en, main])
#main['date']= main['date'].astype(str)
#main.to_csv("main.csv", index = False)

## Merging files

en["date"] = pd.to_datetime(en["date"])
en = en[en["date"] > "2023-03-24"]
en.to_csv("landmine_tweets_eng_processed.csv", index = False)

non_eng["date"] = pd.to_datetime(non_eng["date"])
non_eng = non_eng[non_eng["date"] > "2023-03-24"]
non_eng.to_csv("landmine_tweets_non_eng_processed.csv", index = False)

# Model for English Tweets

In [92]:
# run model for eng tweets --------------------------------------------
model = TransformersNER("tner/bert-large-tweetner7-2020")

# list comprehension to combine all ner records
tner_records = []
for row_dict in en.to_dict(orient="records"):
    for ner_tag_dict in [model.predict([row_dict["tweet"]])]:
        for i in range(len(ner_tag_dict['prediction'][0])):
                results_dict = create_dict(i, ner_tag_dict)
                tner_records.append({**row_dict, **results_dict})


# create ner results df
tner_df = create_ner_tag_df_en(tner_records, "tner/bert-large-tweetner7-2020")
tner_df['tagged_text'] = tner_df['tagged_text'].fillna(" ")

2023-04-18 17:39:03 INFO     initialize language model with `tner/bert-large-tweetner7-2020`
2023-04-18 17:39:06 INFO     use CRF
2023-04-18 17:39:06 INFO     loading pre-trained CRF layer
2023-04-18 17:39:06 INFO     label2id: {'B-corporation': 0, 'B-creative_work': 1, 'B-event': 2, 'B-group': 3, 'B-location': 4, 'B-person': 5, 'B-product': 6, 'I-corporation': 7, 'I-creative_work': 8, 'I-event': 9, 'I-group': 10, 'I-location': 11, 'I-person': 12, 'I-product': 13, 'O': 14}
2023-04-18 17:39:06 INFO     device   : cpu
2023-04-18 17:39:06 INFO     gpus     : 0
2023-04-18 17:39:07 INFO     encode all the data: 1
100%|██████████| 1/1 [00:00<00:00,  2.81it/s]
2023-04-18 17:39:07 INFO     encode all the data: 1
100%|██████████| 1/1 [00:00<00:00,  3.38it/s]
2023-04-18 17:39:07 INFO     encode all the data: 1
100%|██████████| 1/1 [00:00<00:00,  2.61it/s]
2023-04-18 17:39:08 INFO     encode all the data: 1
100%|██████████| 1/1 [00:00<00:00,  3.39it/s]
2023-04-18 17:39:08 INFO     encode all the 

In [93]:
# function to combine ner tags
def combine_ner_tags(df, spacer):
    """
    Function to recombine NER tags from B- to I-
    df = dataframe
    spacer = what kind of spacing to use to combine text. Use "" for non english and " " for english
    """
    combined_tokens = []
    df = df.reset_index()
    if df.shape[0] !=0:
        while df.iloc[0]["ner_tag"].startswith("I-"):
            df=df.iloc[1:,]
        df = df.reset_index()
        cur_tokens = [df.loc[0, "tagged_text"]]
        for i in range(1, len(df)):
            if df.loc[i, "ner_tag"].startswith("B-"):
                combined_tokens.append(cur_tokens)
                cur_tokens = [df.loc[i, "tagged_text"]]
            elif df.loc[i, "ner_tag"].startswith("I-"):
                 cur_tokens.append(df.loc[i, "tagged_text"])
        b_tags = df[df["ner_tag"].str.startswith("B-")]
        combined_tokens.append([b_tags.tail(1)["tagged_text"].values[0]])
        # update the combined text join based on which language. no space for non_eng
        combined_text = [spacer.join(i) for i in combined_tokens]
        b_tags["tagged_text"] = combined_text
        return b_tags
    else:
        print("No non-english tags")
    

In [94]:
# recombine ner tags -----------------------------------------------------------------------

# recombine NER tags
tner_df_combined = combine_ner_tags(tner_df, " ")

tner_df_combined.drop(columns = ["level_0", "index"], axis = 1, inplace = True)
# write to csv
#tner_df_combined.to_csv("landmine_tweets_ner_results_eng_clean.csv", index = False)

/tmp/ipykernel_1184/3058398688.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  b_tags["tagged_text"] = combined_text


In [109]:
tner_df_combined.to_csv("landmine_tweets_ner_results_eng_clean.csv", index = False)

In [97]:
tner_df_combined.tail()

,document_id,sentence_number,sentence_text,tagged_text,ner_tag,tagged_text_loci,ner_model,probability
6381,1647962268687106052,1,rt landmines are not planted in the village wh...,village,B-location,22,tner/bert-large-tweetner7-2020,0.994553
6382,1647953818754367488,1,landmines are not planted in the village while...,sher ali,B-person,11,tner/bert-large-tweetner7-2020,0.999994
6384,1647953818754367488,1,landmines are not planted in the village while...,village,B-location,21,tner/bert-large-tweetner7-2020,0.995906
6385,1647937127001325569,1,ptm is attempting to deceive the public by cla...,sher ali,B-person,20,tner/bert-large-tweetner7-2020,0.999995
6387,1647937127001325569,1,ptm is attempting to deceive the public by cla...,community,B-group,28,tner/bert-large-tweetner7-2020,0.979364


# Model for non english tweets

In [110]:
# run model for non eng tweets --------------------------------------------
if non_eng.shape[0]!=0:

    tokenizer = AutoTokenizer.from_pretrained("Davlan/bert-base-multilingual-cased-ner-hrl")
    model = AutoModelForTokenClassification.from_pretrained("Davlan/bert-base-multilingual-cased-ner-hrl")
    nlp = pipeline("ner", model=model, tokenizer=tokenizer)

    # apply ner model for non eng tweets
    non_eng["ner"] = non_eng["tweet"].apply(nlp)

    # list comprehension to combine all ner records
    ner_records = []
    for row_dict in non_eng.to_dict(orient="records"):
        for ner_tag_dict in nlp(row_dict["tweet"]):  # I Don't recall exact ner model function call
            ner_records.append({**row_dict, **ner_tag_dict})

    # create ner tag df
    ner_df = create_ner_tag_df_multi(ner_records, "Davlan/bert-base-multilingual-cased-ner-hrl")

    # removing all English characters from ner tag df because the multilingual model isn't great for English text

    # detect language of ner tags
    ner_df["language"]= ner_df["tagged_text"].apply(isEnglish)

    # drop English rows
    ner_df= ner_df[ner_df["language"] == False].drop(columns="language", axis=True)

    # cleaning text
    ner_df['tagged_text'] = ner_df['tagged_text'].fillna(" ")
    ner_df['tagged_text'] = ner_df["tagged_text"].str.replace("##", "")

    # recombine NER tags
    ner_df_combined = combine_ner_tags(ner_df, "")
    ner_df_combined.drop(columns = ["level_0", "index"], axis = 1, inplace = True)


    # write to csv
    #ner_df_combined.to_csv("landmine_tweets_ner_results_non_eng_clean.csv", index = False)

/tmp/ipykernel_1184/3058398688.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  b_tags["tagged_text"] = combined_text


In [111]:
ner_df_combined = combine_ner_tags(ner_df, "")
ner_df_combined.drop(columns = ["level_0", "index"], axis = 1, inplace = True)

/tmp/ipykernel_1184/3058398688.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  b_tags["tagged_text"] = combined_text


In [112]:
ner_df_combined.to_csv("landmine_tweets_ner_results_non_eng_clean.csv", index = False)

## Recombining NER tags for the days it was missed

In [102]:
query =("SELECT * FROM ner_tagged_landmine_tweets")

In [103]:
df = pd.read_sql_query(query, cnx)

In [105]:
df.shape

(24599, 8)

In [106]:
df_clean = df.drop_duplicates(subset=['document_id', 'tagged_text'], keep="first")

In [107]:
df_clean.shape

(24104, 8)

In [108]:
df_clean.to_csv("tner_results_combined_clean.csv", index = False)